In [ ]:
import os, pathlib, json, shutil

notebooks_dir = pathlib.Path(os.getcwd())


In [ ]:
sample_extract = notebooks_dir / 'sample_extract'

target_dir = notebooks_dir.parent / 'snowflake/public-docs'
syntax_dir = target_dir / 'syntax'
query_dir = target_dir / 'queries'

shutil.rmtree(query_dir)
shutil.rmtree(syntax_dir)

with (sample_extract / 'snowflake-raw.json').open('r') as f:
    examples = json.load(f)
examples = sorted(examples, key=lambda _: _['example'])
queries = []
syntax = []
for e in examples:
    url = e['url']
    example = e['example']
    item = (url, example)

    is_query = ';' in example and '-- NOT VALID' not in example
    if example[0] == "'":
        # 'SOMETHING%' ILIKE '%\\%%' ESCAPE '\\';
        continue
    if example[0] == "{":
        # 'SOMETHING%' ILIKE '%\\%%' ESCAPE '\\';
        continue
    if ':=' in example and '$$' not in example:
        is_query = False
    if '] ;' in example:
        is_query = False
    if '<whitespace>' in example:
        is_query = False
    if '...' in example:
        is_query = False
        if "'...'" in example:
            is_query = True
        if "-- .." in example:
            is_query = True
    if is_query:
        lines = []
        for line in example.splitlines():
            if len(line) > 0 and ((line[0] == '+' or line[0] == '|') 
                                  # sometimes tables are with invalid formatting
                                  or (line[-1] == '+' or line[-1] == '|')):
                # table results
                continue
            lines.append(line)
        example = "\n".join(lines)
        if len(example) == 0:
            continue
        item = (url, example)
    (queries if is_query else syntax).append(item)

for i, (url, example) in enumerate(syntax):
    padded = f'{i:04}'
    filename = syntax_dir / padded[0:2] / padded[0:3] / f'{padded}.md'
    filename.parent.mkdir(0o755, True, True)
    print(f'SYNTAX:{example}')
    with filename.open('w+') as f:
        f.write(f'''See [{url}]({url})
```
{example}
```
''')

for i, (url, example) in enumerate(queries):
    padded = f'{i:04}'
    filename = query_dir / padded[0:2] / padded[0:3] / f'{padded}.sql'
    filename.parent.mkdir(0o755, True, True)
    # print(f"QUERY:\n{example}\n")
    with filename.open('w+') as f:
        f.write(f"-- see {url}\n\n{example}")
